In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from datacleaner import *

/Users/victoriawu/opt/anaconda3/envs/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

In [51]:
df = get_dataframe()

The number of negative values in target_HF column is: 12479
The number of positive values in target_HF column is: 7917


In [57]:
#Count the number of diagnosis risk factors
icd9_cols = df.filter(regex='^icd9').columns

# Find all columns with age of diagnoses
age_cols = df.filter(regex='admit_age').columns
age_cols = age_cols[:-1] #don't get the HF age cause we don't want to drop that one

# Find all columns with icu stay of diagnoses
icu_cols = df.filter(regex='icu_stay').columns

# Find columns associated with echo data
echo_cols = ['height', 'weight', 'bpsys', 'bpdias', 'hr', 'EF']

In [58]:
grouped_df = pd.DataFrame()

for i in range(len(icd9_cols)):
    # stack the three columns into a single list column
    df['new_'+icd9_cols[i]] = df.apply(lambda x: [x[icd9_cols[i]], x[age_cols[i]], x[icu_cols[i]]], axis=1)
    grouped_df['new_'+icd9_cols[i]] = (df['new_'+icd9_cols[i]])
    df.drop(('new_'+icd9_cols[i]), axis=1)

In [59]:
#group all echo variables with age and echo_icu_stay
for i in range(len(echo_cols)):
    # stack the three columns into a single list column
    df['new_'+echo_cols[i]] = df.apply(lambda x: [x[echo_cols[i]], x['age'], x['echo_icu_stay']], axis=1)
    grouped_df['echo_'+echo_cols[i]] = (df['new_'+echo_cols[i]])
    df.drop('new_'+echo_cols[i], axis=1)

In [65]:
#add gender - standalone

# replace 'M' and 'F' with 1 and 2, respectively
df['gender'] = df['gender'].replace({'M': 1, 'F': 2})
grouped_df['gender'] = df.apply(lambda x: [x['gender'], 0, 0], axis=1)

#add target HF
grouped_df['HF'] = df['target_HF']

In [66]:
grouped_df

,new_icd9_hypertension,new_icd9_myocardialinfarction,new_icd9_diabetesmellitus,new_icd9_renalfailure,new_icd9_aorticvalvedisorders,new_icd9_amyloidosis,new_icd9_cancer,new_icd9_thyroiddisorder,new_icd9_conduction,new_icd9_irondeficiency,...,new_icd9_chagas,new_icd9_endomycardialfibrosis,echo_height,echo_weight,echo_bpsys,echo_bpdias,echo_hr,echo_EF,gender,HF
0,"[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[1.0, 61.42, 1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[1.0, 54.97, 1.0]",...,"[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[67.0, 54.97, 1]","[125.0, 54.97, 1]","[120.0, 54.97, 1]","[80.0, 54.97, 1]","[86.0, 54.97, 1]","[55.0, 54.97, 1]","[2, 0, 0]",1.0
1,"[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[1.0, 79.54, 1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]",...,"[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[66.0, 79.54, 1]","[150.0, 79.54, 1]","[134.0, 79.54, 1]","[61.0, 79.54, 1]","[60.0, 79.54, 1]","[35.0, 79.54, 1]","[2, 0, 0]",0.0
2,"[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]",...,"[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[68.0, 54.78, 1]","[220.0, 54.78, 1]","[149.0, 54.78, 1]","[90.0, 54.78, 1]","[60.0, 54.78, 1]","[55.0, 54.78, 1]","[1, 0, 0]",0.0
3,"[1.0, 52.37, 1.0]","[1.0, 52.37, 1.0]","[1.0, 52.37, 1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]",...,"[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[-1.0, 52.4, 1]","[-1.0, 52.4, 1]","[-1.0, 52.4, 1]","[-1.0, 52.4, 1]","[-1.0, 52.4, 1]","[55.0, 52.4, 1]","[1, 0, 0]",0.0
6,"[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[1.0, 87.3, 1.0]","[1.0, 87.3, 1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]",...,"[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[71.0, 87.3, 1]","[90.0, 87.3, 1]","[130.0, 87.3, 1]","[70.0, 87.3, 1]","[40.0, 87.3, 1]","[50.0, 87.3, 1]","[1, 0, 0]",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39173,"[1.0, 66.97, 1.0]","[0.0, -1.0, -1.0]","[1.0, 66.97, 1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[1.0, 66.97, 1.0]",...,"[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[66.0, 66.97, 1]","[182.0, 66.97, 1]","[149.0, 66.97, 1]","[86.0, 66.97, 1]","[76.0, 66.97, 1]","[20.0, 66.97, 1]","[2, 0, 0]",1.0
39175,"[1.0, 53.39, 1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]",...,"[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[64.0, 53.39, 1]","[136.0, 53.39, 1]","[126.0, 53.39, 1]","[67.0, 53.39, 1]","[75.0, 53.39, 1]","[55.0, 53.39, 1]","[2, 0, 0]",1.0
39176,"[1.0, 80.73, 1.0]","[1.0, 80.73, 1.0]","[1.0, 80.45, 1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[1.0, 80.73, 1.0]","[0.0, -1.0, -1.0]",...,"[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[61.0, 80.55, 1]","[144.0, 80.55, 1]","[160.0, 80.55, 1]","[65.0, 80.55, 1]","[60.0, 80.55, 1]","[50.0, 80.55, 1]","[2, 0, 0]",1.0
39181,"[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[1.0, 65.71, 1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[1.0, 65.71, 1.0]","[0.0, -1.0, -1.0]",...,"[0.0, -1.0, -1.0]","[0.0, -1.0, -1.0]","[64.0, 65.72, 1]","[191.0, 65.72, 1]","[100.0, 65.72, 1]","[40.0, 65.72, 1]","[-1.0, 65.72, 1]","[55.0, 65.72, 1]","[2, 0, 0]",0.0


In [74]:
# custom dataset
class MIMICDataset(Dataset):
    def __init__(self, df):
        self.data = df.values.tolist()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label = (self.data[idx][-1])
        patient = (self.data[idx][:-1])
        sample = {'patient': torch.FloatTensor(patient), 'label': torch.Tensor([label])} #torch.FloatTensor(self.data[idx])
        return sample

# create a dataloader
batch_size = 2
dataset = MIMICDataset(grouped_df)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)